In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.7.2/'

In [3]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [4]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_hPa_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_hPa_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time = [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [5]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [6]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [7]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [8]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [9]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [10]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [11]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [12]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [13]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [14]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [15]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [16]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [17]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [18]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [19]:
Control_Compo[Control_Compo["ABC"] == "A"]

,Orig Index,ABC,Time,Lon,Lat,PathID
0,0,A,1985-06-19 12:00:00,-82.19,27.99,0
1,1,A,1985-06-19 18:00:00,-82.26,27.79,0
2,2,A,1985-06-20 00:00:00,-82.33,27.58,0
3,3,A,1985-06-20 06:00:00,-82.40,27.38,0
4,4,A,1985-06-20 12:00:00,-82.46,28.31,0
...,...,...,...,...,...,...
10154,10154,A,2014-11-11 06:00:00,-76.86,26.37,306
10155,10155,A,2014-11-11 12:00:00,-75.85,27.61,306
10156,10156,A,2014-11-11 18:00:00,-74.54,29.19,306
10157,10157,A,2014-11-12 00:00:00,-73.07,32.06,306


In [20]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [21]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [22]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    Code_List = DF_Storm["Code"]
    Name_List = DF_Storm["Name"]
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Code": Code_List, "Name": Name_List, \
    "Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP": SLP, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [23]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [24]:
# Find Total Precip Rate From Compo File
def Precip(Compo_nc, Compo_Index):
    Precip_ms = numpy.array(Compo_nc.snap_PRECT[int(Compo_Index)])
    Precip_mmhr = Precip_ms * 3600 * 1000
    return (Precip_mmhr)

In [25]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    SLP = DF_Storm_Compo["SLP"]
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((5,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
# Count Number of Data Points With Windspeed Above 18,25,33m/s
        Snap_Sort = numpy.sort(Snap_850.ravel())
        Count_18 = len(Snap_Sort[Snap_Sort >= 18])
        Count_25 = len(Snap_Sort[Snap_Sort >= 25])
        Count_33 = len(Snap_Sort[Snap_Sort >= 33])
        Wind_Field_Info[2][k] = Count_18
        Wind_Field_Info[3][k] = Count_25
        Wind_Field_Info[4][k] = Count_33
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    DF_Storm_Compo["18m/s"] = Wind_Field_Info[2]
    DF_Storm_Compo["25m/s"] = Wind_Field_Info[3]
    DF_Storm_Compo["33m/s"] = Wind_Field_Info[4]
    return (DF_Storm_Compo)

In [26]:
# Find Max Precip and Total Precip Over Area
def Precip_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
#
# Create Array to Store Data
    Precip_Field_Info = numpy.zeros((5,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Precip Snap From Compo_nc
        Snap_Precip = Precip(Compo_nc, Compo_Index[k])
# Find Maximum Precip Rate
        Max_Precip = numpy.max(Snap_Precip)
        Precip_Field_Info[0][k] = Max_Precip
# Find Areal Precip Total
        Snap_Sort = numpy.sort(Snap_Precip.ravel())
        Areal_Precip_Total = numpy.sum(Snap_Sort)
        Precip_Field_Info[1][k] = Areal_Precip_Total
# Count Number of Data Points With Precip Rate Above 0.5, 1, 5mm/hr
        Count_05 = len(Snap_Sort[Snap_Sort >= 0.5])
        Count_1 = len(Snap_Sort[Snap_Sort >= 1])
        Count_5 = len(Snap_Sort[Snap_Sort >= 5])
        Precip_Field_Info[2][k] = Count_05
        Precip_Field_Info[3][k] = Count_1
        Precip_Field_Info[4][k] = Count_5
# 
# Add Precip Field Info Into DF Storm Compo
    DF_Storm_Compo["Max Precip Rate"] = Precip_Field_Info[0]
    DF_Storm_Compo["Areal Precip Total"] = Precip_Field_Info[1]
    DF_Storm_Compo["0.5mm/hr"] = Precip_Field_Info[2]
    DF_Storm_Compo["1mm/hr"] = Precip_Field_Info[3]
    DF_Storm_Compo["5mm/hr"] = Precip_Field_Info[4]
    return (DF_Storm_Compo)

In [27]:
# Create New Data DF
def DF_Data_Compo(Data_DF, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
    ABC_List = ET_DF["ABC"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Composite_Data(Code_List[n], Data_DF, Compo_DF)
# Find Which Compo nc To Use
        if ABC_List[n] == "A":
            Compo_nc = Compo_nc_A
        elif ABC_List[n] == "B":
            Compo_nc = Compo_nc_B
        elif ABC_List[n] == "C":
            Compo_nc = Compo_nc_C
# Apply Functions For Finding Wind Field and Precip Field
        DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Precip_Field_Find(DF_Storm_Compo, Compo_nc)
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0:
            if DF_Storm_Compo["Storm Phase"][len(DF_Storm_Compo)-1] == "Extratropical":
# Combine DF Storm Compos
                try:
                    Data_Compo = pandas.concat([Data_Compo, DF_Storm_Compo])
                except:
                    Data_Compo = DF_Storm_Compo.copy()
    Data_Compo_Final = Data_Compo.reset_index().drop("index", axis=1)
    return (Data_Compo_Final)

In [28]:
# Create New ET DF
def DF_ET_Compo(Data_Compo, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        ET_Storm = Find_Storm(ET_DF, Code_List[n])
        DF_Storm_Compo = Find_Storm(Data_Compo, Code_List[n])
# Find ET Begin and ET Complete Time
        Begin_Time = ET_Storm["ET Begin Time"][0]
        Compl_Time = ET_Storm["ET Complete Time"][0]
        DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
        DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0 and len(DF_Begin) > 0 and len(DF_Compl) > 0:
# Combine ET Storm Compos
            ET_Storm_Compo = Find_ET_Compo(Code_List[n], ET_Storm, DF_Begin, DF_Compl)
            try:
                ET_Compo = pandas.concat([ET_Compo, ET_Storm_Compo])
            except:
                ET_Compo = ET_Storm_Compo.copy()
    ET_Compo_Final = ET_Compo.reset_index().drop("index", axis=1)
    return (ET_Compo_Final)

In [29]:
def Find_ET_Compo(Code, ET_Storm, DF_Begin, DF_Compl):
    ET_Storm_Compo = ET_Storm[["Code", "Name", "ET Begin Time", "ET Complete Time", \
    "ET Begin SLP", "ET Complete SLP"]].copy()
    Vars = ["850hPa Winds", "18m/s", "25m/s", "33m/s", \
    "Max Precip Rate", "Areal Precip Total", "0.5mm/hr", "1mm/hr", "5mm/hr"]
    for m in range(len(Vars)):
        Var = Vars[m]
        Begin_Var = str("ET Begin " + Var)
        Compl_Var = str("ET Complete " + Var)
        ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
        ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
    return (ET_Storm_Compo)

In [30]:
Control_Data_Compo = DF_Data_Compo(Control_Data, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)
Control_Data_Compo.head()

,Code,Name,Compo Index,Time,Lon,Lat,SLP,Storm Phase,850hPa Winds,18m/s,25m/s,33m/s,Max Precip Rate,Areal Precip Total,0.5mm/hr,1mm/hr,5mm/hr
0,TC190001,Alex,0.0,1900-06-19 12:00:00,-82.19,27.99,1015.45,Tropical,23.692062,149.0,0.0,0.0,11.405020,822.940491,407.0,155.0,8.0
1,TC190001,Alex,1.0,1900-06-19 18:00:00,-82.26,27.79,1015.45,Tropical,20.203150,69.0,0.0,0.0,12.442551,886.791443,406.0,141.0,3.0
2,TC190001,Alex,2.0,1900-06-20 00:00:00,-82.33,27.58,1015.45,Tropical,19.720341,23.0,0.0,0.0,2.922493,760.727722,413.0,83.0,0.0
3,TC190001,Alex,3.0,1900-06-20 06:00:00,-82.40,27.38,1015.45,Tropical,20.409258,21.0,0.0,0.0,9.133479,687.665039,404.0,118.0,9.0
4,TC190001,Alex,4.0,1900-06-20 12:00:00,-82.46,28.31,1014.22,Tropical,20.912153,13.0,0.0,0.0,11.239372,660.232849,342.0,111.0,12.0


In [31]:
Control_ET_Compo = DF_ET_Compo(Control_Data_Compo, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)
Control_ET_Compo

,Code,Name,ET Begin Time,ET Complete Time,ET Begin SLP,ET Complete SLP,ET Begin 850hPa Winds,ET Complete 850hPa Winds,ET Begin 18m/s,ET Complete 18m/s,...,ET Begin Max Precip Rate,ET Complete Max Precip Rate,ET Begin Areal Precip Total,ET Complete Areal Precip Total,ET Begin 0.5mm/hr,ET Complete 0.5mm/hr,ET Begin 1mm/hr,ET Complete 1mm/hr,ET Begin 5mm/hr,ET Complete 5mm/hr
0,TC190001,Alex,1900-07-03 18:00:00,1900-07-03 18:00:00,1015.57,1015.57,20.221558,20.221558,59.0,59.0,...,6.990595,6.990595,1069.774048,1069.774048,581.0,581.0,226.0,226.0,8.0,8.0
1,TC190003,Colin,1900-09-18 00:00:00,1900-09-20 12:00:00,952.06,979.68,58.612934,35.713459,370.0,1104.0,...,48.978561,19.580704,1741.779907,1438.482666,748.0,573.0,386.0,335.0,40.0,48.0
2,TC190004,Dawn,1900-11-05 12:00:00,1900-11-06 12:00:00,954.74,992.74,57.812984,38.288841,168.0,628.0,...,68.715897,14.409717,1685.475464,1489.733276,802.0,548.0,290.0,368.0,30.0,44.0
3,TC190005,Ernest,1900-11-06 00:00:00,1900-11-08 00:00:00,996.02,998.22,42.109539,39.965668,642.0,780.0,...,15.045424,6.681671,1277.432861,1194.898193,458.0,690.0,277.0,372.0,29.0,10.0
4,TC190102,Harrison,1901-05-28 12:00:00,1901-05-30 18:00:00,991.56,1008.78,38.040359,15.223821,140.0,0.0,...,33.126488,1.490861,732.234741,73.670769,266.0,12.0,81.0,1.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,TC198802,Micah,1988-02-06 00:00:00,1988-02-06 06:00:00,999.63,999.02,31.929470,33.058334,516.0,416.0,...,7.181813,4.432220,1260.450195,1088.528687,597.0,565.0,295.0,218.0,13.0,0.0
276,TC198805,Seldoen,1988-09-07 06:00:00,1988-09-08 12:00:00,984.68,987.91,40.055161,43.946793,544.0,869.0,...,17.339502,27.767605,1474.180176,1552.025024,531.0,503.0,332.0,327.0,58.0,57.0
277,TC198808,Alice,1988-09-12 06:00:00,1988-09-12 12:00:00,1009.08,1008.35,32.628975,30.829254,745.0,690.0,...,17.701586,12.395168,1233.788696,1091.342163,489.0,420.0,319.0,265.0,28.0,31.0
278,TC198809,Bruce,1988-09-21 06:00:00,1988-09-27 00:00:00,961.07,1001.36,56.035839,36.065895,242.0,699.0,...,39.931343,10.432244,1670.338745,1922.024048,625.0,760.0,278.0,524.0,36.0,71.0


In [32]:
RCP45_Data_Compo = DF_Data_Compo(RCP45_Data, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)
#RCP45_Data_Compo

In [33]:
RCP45_ET_Compo = DF_ET_Compo(RCP45_Data_Compo, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)
RCP45_ET_Compo

,Code,Name,ET Begin Time,ET Complete Time,ET Begin SLP,ET Complete SLP,ET Begin 850hPa Winds,ET Complete 850hPa Winds,ET Begin 18m/s,ET Complete 18m/s,...,ET Begin Max Precip Rate,ET Complete Max Precip Rate,ET Begin Areal Precip Total,ET Complete Areal Precip Total,ET Begin 0.5mm/hr,ET Complete 0.5mm/hr,ET Begin 1mm/hr,ET Complete 1mm/hr,ET Begin 5mm/hr,ET Complete 5mm/hr
0,TC200004,Dawn,2000-03-16 18:00:00,2000-03-18 00:00:00,995.91,984.01,36.465290,40.586483,789.0,1822.0,...,17.680902,16.822504,1144.861694,1936.767090,570.0,892.0,246.0,564.0,23.0,58.0
1,TC200013,Phil,2000-10-25 06:00:00,2000-10-25 18:00:00,996.31,999.55,22.417723,23.703508,318.0,439.0,...,6.389364,5.775116,805.416687,920.012207,462.0,500.0,251.0,287.0,11.0,4.0
2,TC200105,Brandon,2001-09-29 00:00:00,2001-09-29 12:00:00,966.21,989.30,48.570816,36.842369,650.0,872.0,...,45.693920,21.696259,2302.938477,1583.052490,532.0,569.0,339.0,348.0,99.0,71.0
3,TC200203,Geoffrey,2002-10-31 06:00:00,2002-11-01 12:00:00,953.59,1006.73,70.515007,33.755402,1369.0,535.0,...,33.545174,5.291304,1830.107666,829.349915,542.0,377.0,273.0,253.0,78.0,9.0
4,TC200302,Katrina,2003-03-04 00:00:00,2003-03-05 18:00:00,981.00,996.69,31.331041,40.654629,1691.0,2812.0,...,8.016044,7.914567,1716.159058,1157.615356,889.0,546.0,512.0,297.0,11.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,TC209010,Mary,2090-10-17 00:00:00,2090-10-17 06:00:00,989.48,986.06,35.104877,39.669765,1316.0,1923.0,...,16.227900,19.488230,2721.935791,3074.122070,1232.0,1374.0,664.0,703.0,92.0,94.0
176,TC209011,Nabil,2090-11-06 00:00:00,2090-11-09 12:00:00,994.31,970.20,35.673756,41.475708,498.0,2128.0,...,30.660120,9.045152,2282.866455,1442.920044,951.0,702.0,547.0,324.0,81.0,25.0
177,TC209101,Rosalina,2091-01-14 00:00:00,2091-01-15 12:00:00,978.54,968.11,44.941948,43.895290,1390.0,2359.0,...,27.929419,7.418318,2622.375488,1131.618530,1077.0,530.0,488.0,348.0,115.0,17.0
178,TC209105,Alberto,2091-09-06 00:00:00,2091-09-06 12:00:00,996.84,1005.89,34.104332,30.896976,706.0,835.0,...,17.592691,7.426961,663.328125,495.729248,219.0,209.0,136.0,141.0,29.0,9.0


In [34]:
RCP85_Data_Compo = DF_Data_Compo(RCP85_Data, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)
#RCP85_Data_Compo

In [35]:
RCP85_ET_Compo = DF_ET_Compo(RCP85_Data_Compo, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)
RCP85_ET_Compo

,Code,Name,ET Begin Time,ET Complete Time,ET Begin SLP,ET Complete SLP,ET Begin 850hPa Winds,ET Complete 850hPa Winds,ET Begin 18m/s,ET Complete 18m/s,...,ET Begin Max Precip Rate,ET Complete Max Precip Rate,ET Begin Areal Precip Total,ET Complete Areal Precip Total,ET Begin 0.5mm/hr,ET Complete 0.5mm/hr,ET Begin 1mm/hr,ET Complete 1mm/hr,ET Begin 5mm/hr,ET Complete 5mm/hr
0,TC210003,Colin,2100-03-03 18:00:00,2100-03-04 00:00:00,970.11,971.02,41.583744,38.639641,4377.0,4096.0,...,11.060635,9.564248,2040.884399,2037.811157,830.0,924.0,374.0,370.0,43.0,33.0
1,TC210005,Ernest,2100-09-01 06:00:00,2100-09-01 12:00:00,1003.30,1001.84,31.986475,30.011461,416.0,410.0,...,36.507378,30.047020,1222.030884,1687.698853,443.0,463.0,184.0,265.0,43.0,80.0
2,TC210103,Lydia,2101-07-26 12:00:00,2101-07-28 00:00:00,971.89,992.45,52.436172,33.237667,767.0,703.0,...,31.085773,10.253569,1769.748413,1272.197754,529.0,457.0,293.0,329.0,88.0,56.0
3,TC210105,Nicole,2101-09-12 18:00:00,2101-09-14 18:00:00,934.13,962.56,70.148903,42.286533,698.0,1530.0,...,47.622822,19.295294,1808.515381,1250.616333,360.0,408.0,204.0,269.0,60.0,52.0
4,TC210106,Phil,2101-09-22 18:00:00,2101-09-23 18:00:00,997.21,1001.49,39.054733,40.776558,169.0,891.0,...,33.887508,20.221531,1583.632202,1311.796875,622.0,448.0,298.0,243.0,45.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,TC219005,Ollie,2190-09-12 06:00:00,2190-09-13 06:00:00,1008.13,993.61,23.499449,31.749573,38.0,115.0,...,25.182686,49.256500,2288.108154,2473.934082,815.0,1110.0,517.0,655.0,84.0,57.0
195,TC219007,Sergio,2190-11-17 12:00:00,2190-11-20 12:00:00,920.27,981.25,82.263176,46.242245,711.0,1700.0,...,87.822502,11.376763,3711.232910,1845.116333,809.0,707.0,337.0,472.0,99.0,67.0
196,TC219008,Toby,2190-11-17 06:00:00,2190-11-18 12:00:00,978.43,967.20,51.803890,50.406021,1031.0,2788.0,...,31.058935,17.317783,2119.778564,3132.332764,685.0,951.0,390.0,731.0,81.0,159.0
197,TC219102,Alex,2191-09-09 12:00:00,2191-09-11 00:00:00,926.25,984.72,73.247314,44.278305,702.0,978.0,...,63.574150,23.566832,2746.804443,1853.517456,946.0,430.0,429.0,325.0,81.0,106.0
